In [1]:
cd ..

/Users/wonjunekang/Desktop/MAS.S62/project/conversation-quality


In [2]:
import os
import json
import csv
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler

from models import MultipleRegressionNet

In [3]:
lvn_data_csv = "./data/csv/lvn_metrics.csv"
drop_feats=["Conversation ID", "Mean inter-speaker silence", "Interruption rate"]

lvn_data_df = pd.read_csv(lvn_data_csv, delimiter='\t').drop(drop_feats, axis=1)
lvn_data_df

,Words per hour,Speech per turn,Turn taking balance,Grade level,MATTR lexical diversity,Mean word length,VADER sentiment,Responsivity rate
0,9180.18,18.21,0.876,4.32,422,4.40,0.352,0.205
1,9163.48,34.91,0.982,5.23,450,4.39,0.356,0.136
2,9069.23,28.93,0.954,5.28,421,4.31,0.399,0.159
3,8627.24,20.12,0.866,4.62,430,4.36,0.430,0.237
4,9005.07,42.85,0.976,5.51,409,4.28,0.597,0.133


In [4]:
keeper_train_csv = "./data/csv/keeper_metrics_full.csv"
keeper_train_df = pd.read_csv(keeper_train_csv, delimiter='\t').drop(drop_feats, axis=1)
keeper_train_df

,Words per hour,Speech per turn,Turn taking balance,Grade level,MATTR lexical diversity,Mean word length,VADER sentiment,Responsivity rate
0,6373.99,9.02,0.966,2.61,444,4.28,0.299,0.152
1,6493.69,9.42,0.959,3.08,406,4.33,0.294,0.197
2,5119.84,8.54,0.938,2.55,374,4.30,0.262,0.132
3,7828.87,5.62,0.983,0.96,446,4.22,0.185,0.137
4,6318.28,8.19,0.956,1.41,407,4.23,0.233,0.211
5,7501.94,7.71,0.975,1.97,425,4.25,0.265,0.226
6,10473.77,7.62,0.877,3.81,440,4.28,0.274,0.081
7,11367.38,6.00,0.873,3.03,439,4.22,0.204,0.089
8,9562.37,9.58,0.857,3.61,418,4.21,0.247,0.082
9,6191.68,6.60,0.990,1.58,416,4.31,0.272,0.086


In [5]:
keeper_labels_csv = "./data/csv/keeper_survey_avg.csv"
keeper_labels_df = pd.read_csv(keeper_labels_csv).drop(["treatment", "conversation"], axis=1)
keeper_labels_df

,ease,sp,anxiety,tone
0,3.500000,1.642857,2.500000,2.781250
1,3.166667,1.535714,2.000000,1.937500
2,4.000000,2.571429,2.500000,2.729167
3,2.666667,1.571429,2.416667,2.125000
4,3.000000,1.964286,2.125000,2.156250
5,2.777778,1.714286,1.833333,2.104167
6,2.777778,1.428571,1.666667,2.083333
7,2.000000,2.000000,1.800000,1.812500
8,1.833333,1.535714,1.500000,1.578125
9,3.166667,1.428571,2.000000,1.875000


In [6]:
data_scaler = StandardScaler()
train_data_scaled = data_scaler.fit_transform(keeper_train_df.to_numpy())
lvn_data_scaled = data_scaler.transform(lvn_data_df.to_numpy())

In [7]:
labels_scaler = StandardScaler()
labels_scaled = labels_scaler.fit_transform(keeper_labels_df.to_numpy())

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda:0"):
    print("Using GPU.")
else:
    print("Using CPU.")

Using CPU.


In [13]:
ckpt_dir = './checkpoints/mrn_keeper_part2'

pred_450 = []
pred_451 = []
pred_452 = []
pred_453 = []
pred_454 = []

for ckpt in sorted(os.listdir(ckpt_dir)):
    path_to_ckpt = os.path.join(ckpt_dir, ckpt)
    model = MultipleRegressionNet(n_feats=8, n_hidden=10)
    model.load_state_dict(torch.load(path_to_ckpt))
    model = model.to(device)
    model.eval()
#     print(f"Loaded model from {path_to_ckpt}")

    inp = torch.tensor(lvn_data_scaled, dtype=torch.float)
    out = model(inp).cpu().detach().numpy()
    out_unscaled = labels_scaler.inverse_transform(out)
    
    pred_450.append(out_unscaled[0].tolist())
    pred_451.append(out_unscaled[1].tolist())
    pred_452.append(out_unscaled[2].tolist())
    pred_453.append(out_unscaled[3].tolist())
    pred_454.append(out_unscaled[4].tolist())

In [15]:
pred_path = './data/lvn/csv_preds_part2_model'
filenames = ['lvn_450', 'lvn_451', 'lvn_452', 'lvn_453', 'lvn_454']
all_pred_vals = [pred_450, pred_451, pred_452, pred_453, pred_454]

for filename, pred_vals in zip(filenames, all_pred_vals):
    pred_file = os.path.join(pred_path, filename+".csv")
    with open(pred_file, 'w', newline='') as f:
        csv_writer = csv.writer(f, delimiter='\t')
        csv_writer.writerow(['ease', 'sp', 'anxiety', 'tone'])
        
        for preds in pred_vals:
            csv_writer.writerow([round(x, 3) for x in preds])
    f.close()